<a href="https://colab.research.google.com/github/awaw24/Metaheurystyki/blob/main/Metaheurystyki_Lab_6_Zad_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Poniższy kod w języku Python zawiera implementację algorytmu CLONALG wraz z przykładowymi badaniami na funkcji Rastrigina (10 wymiarów). Wykorzystane zostały trzy zestawy parametrów (rozmiar populacji, liczba najlepszych osobników, liczba iteracji) natomiast każda z konfiguracji została uruchomiona 5‑krotnie, by uzyskać średnie oraz odchylenia najlepszych wyników.

Kod zawiera:

Definicję funkcji Rastrigina.

Implementację algorytmu CLONALG:
- inicjalizacja
- selekcja klonalna
- klonowanie
- hipermutacja
- dojrzewanie
- wymiana losowa

Pętlę eksperymentalną generującą tabelę wyników (średnie, odchylenia).

Ponizej kodu znajduje się komentarz do uzyskanych przezemnie wyników.

In [1]:
import numpy as np
import pandas as pd

# Funkcje testowe
def rastrigin(x):
    n = len(x)
    return 10*n + np.sum(x**2 - 10*np.cos(2*np.pi*x))

# Implementacja CLONALG
def clonalg(func, bounds, dim, pop_size, n_best, max_iter, random_replace, clone_factor=0.1, mutate_factor=0.5):
    # Inicjalizacja populacji
    pop = np.random.uniform(bounds[0], bounds[1], (pop_size, dim))
    fitness = np.apply_along_axis(func, 1, pop)

    best_history = []
    for it in range(max_iter):
        # Wybór najlepszych n_best
        idx = np.argsort(fitness)
        best_idx = idx[:n_best]
        best = pop[best_idx]
        best_f = fitness[best_idx]
        # Klonowanie proporcjonalnie do pokrewieństwa
        affinities = 1/(1+best_f)
        clones = []
        for i, indiv in enumerate(best):
            n_clones = int(np.ceil(clone_factor * affinities[i] * pop_size))
            clones.extend([indiv.copy() for _ in range(n_clones)])
        clones = np.array(clones)
        # Hipermutacja
        clone_fit = np.apply_along_axis(func, 1, clones)
        for i in range(len(clones)):
            rate = mutate_factor * (1 - affinities[i % n_best])
            clones[i] += rate * np.random.randn(dim)
            clones[i] = np.clip(clones[i], bounds[0], bounds[1])
        # Ocena klonów
        clone_fit = np.apply_along_axis(func, 1, clones)
        # Wybór najlepszych klonów
        combined = np.vstack([pop, clones])
        comb_fit = np.concatenate([fitness, clone_fit])
        idx2 = np.argsort(comb_fit)
        pop = combined[idx2][:pop_size]
        fitness = comb_fit[idx2][:pop_size]
        # Zastąpienie najgorszych losowych rozwiązań
        worst = pop_size - random_replace
        pop[worst:] = np.random.uniform(bounds[0], bounds[1], (random_replace, dim))
        fitness[worst:] = np.apply_along_axis(func, 1, pop[worst:])
        best_history.append(fitness.min())
    return pop, fitness, best_history

# Eksperyment
if __name__ == '__main__':
    dim = 10
    bounds = (-5.12, 5.12)
    func = rastrigin
    # Zestaw parametrów: (pop_size, n_best, iter)
    params = [
        (50, 5, 100),
        (100, 10, 200),
        (200, 20, 300)
    ]
    results = []
    for pop_size, n_best, iters in params:
        runs = []
        for run in range(5):
            pop, fit, hist = clonalg(func, bounds, dim, pop_size, n_best, iters, random_replace=int(0.1*pop_size))
            runs.append(fit.min())
        results.append({
            'pop_size': pop_size,
            'n_best': n_best,
            'iters': iters,
            'mean_best': np.mean(runs),
            'std_best': np.std(runs)
        })
    df = pd.DataFrame(results)
    print(df)


   pop_size  n_best  iters  mean_best   std_best
0        50       5    100  65.893146  13.766196
1       100      10    200  45.026157   8.754860
2       200      20    300  37.650506   6.271854


<br>

**Komentarz do uzyskanych wyników:**

<br>

**Wpływ wielkości populacji i liczby iteracji**

Wraz ze wzrostem rozmiaru populacji (50 → 100 → 200) i liczby iteracji (100 → 200 → 300) obserwujemy istotne obniżenie wartości średniego najlepszego rozwiązania:

- Dla najmniejszego zestawu (50, 5, 100) średnia to ≈ 65.9,

- Dla środkowego (100, 10, 200) — ≈ 45.0,

- Dla największego (200, 20, 300) — ≈ 37.7.
To potwierdza, że większa populacja i dłuższy czas ewolucji dają algorytmowi więcej szans na przeszukanie przestrzeni rozwiązań i lepsze przybliżenie minimum.

<br>

**Redukcja zmienności wyników**

Jednocześnie odchylenie standardowe najlepszych wyników (std_best) systematycznie spada: z ~13.8 dla najmniejszego zestawu, przez ~8.75, aż do ~6.27 dla największego. Mniejsza wariancja wskazuje, że konfiguracje o większej populacji i więcej iteracjach dają nie tylko lepsze, ale też bardziej stabilne rezultaty między kolejnymi powtórzeniami.

<br>

**Malejące zyski**

Chociaż kolejne zwiększanie parametrów poprawia wyniki, przyrost korzyści maleje: różnica w mean_best między zestawem 1 a 2 to ~20.9, a między 2 i 3 – już tylko ~7.4. Sugeruje to, że po pewnym poziomie dalsze powiększanie populacji lub wydłużanie iteracji będzie miało coraz mniejszy wpływ na poprawę jakości rozwiązania.

<br>

**Podsumowanie**

- Jeśli celam jest głównie maksymalizacja jakości należy zastosować większe populacje i więcej iteracji, o ile pozwala na to budżet obliczeniowy.

- Kompromis między czasem a rozwiązaniem, konfiguracja (100, 10, 200) wydaje się rozsądnym punktem startowym: znacząco lepsza od najmniejszej, ale przy relatywnie umiarkowanym czasie obliczeń.

- Testy wskazują, iż większe zasoby (populacja, iteracje) przekładają się na lepsze i bardziej spójne wyniki, ale z malejącym przyrostem zwrotu.